In [0]:
import os

# Define the base path to the landing folder
base_path = "/Volumes/mcutini/synthea/landing/"

# Get all subdirectories in the base path
subdirectories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

# Remove the 'notes' folder from the subdirectories list
subdirectories = [d for d in subdirectories if d != 'notes']

print(subdirectories)

# Iterate over each subdirectory and process the files
for subdir in subdirectories:
    file_path = os.path.join(base_path, subdir)
    
    # Read the files into a DataFrame
    df = spark.read.format("csv").option("header", "true").load(file_path)
    
    # Write the DataFrame to a Delta table
    table_name = f"mcutini.synthea.{subdir}"
    df.write.format("delta").mode("overwrite").saveAsTable(table_name)

In [0]:
import os

# Define the path to the notes folder
notes_path = "/Volumes/mcutini/synthea/synthetic_files_raw/output/notes/"

# Get all files in the notes folder
note_files = [f for f in os.listdir(notes_path) if os.path.isfile(os.path.join(notes_path, f))]

# Create a list to hold the file data
data = []

# Iterate over each file and read its content
for note_file in note_files:
    file_path = os.path.join(notes_path, note_file)
    with open(file_path, 'r') as file:
        file_text = file.read()
        data.append((note_file, file_text))

# Create a DataFrame from the data
notes_df = spark.createDataFrame(data, ["file_name", "file_text"])

# Write the DataFrame to a Delta table
notes_df.write.format("delta").mode("overwrite").saveAsTable("mcutini.synthea.notes")